In [12]:
%pylab inline

import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedShuffleSplit

Populating the interactive namespace from numpy and matplotlib


In [13]:
labels = pd.read_csv("data/trainLabels.csv")

In [26]:
labels2 = labels[:10]
type(labels2.image)

pandas.core.series.Series

In [32]:
import os
labels2.iloc[:,0] = labels2.image.apply(lambda  img: os.path.join("data", "train", "resized", img + ".jpg"))

/home/dudevil/.virtualenvs/drd/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [33]:
labels2

,image,level
0,data/train/resized/data/train/resized/10_left....,0
1,data/train/resized/data/train/resized/10_right...,0
2,data/train/resized/data/train/resized/13_left....,0
3,data/train/resized/data/train/resized/13_right...,0
4,data/train/resized/data/train/resized/15_left....,1
5,data/train/resized/data/train/resized/15_right...,2
6,data/train/resized/data/train/resized/16_left....,4
7,data/train/resized/data/train/resized/16_right...,4
8,data/train/resized/data/train/resized/17_left....,0
9,data/train/resized/data/train/resized/17_right...,1


In [11]:
sss = StratifiedShuffleSplit(labels.level, 1, test_size=0.1, random_state=0)

train_index, valid_index = list(sss).pop()

In [16]:
#labels.iloc[train_index]
labels.image[valid_index]

5810       7320_left
19141    24155_right
24166     30550_left
9463     11913_right
16025    20093_right
29965    37913_right
28981    36695_right
27908     35343_left
17906     22561_left
32022     40438_left
27076     34274_left
213        241_right
26984     34147_left
15784     19789_left
23373    29572_right
...
22970     29020_left
17013    21341_right
5155      6490_right
32142     40593_left
32147    40601_right
14851    18589_right
18563    23403_right
10063    12673_right
30874     39032_left
30554     38634_left
22419    28272_right
10591    13323_right
17447    21922_right
25995    32891_right
5357      6734_right
Name: image, Length: 3512, dtype: object

In [2]:
from load_dataset import DataLoader

ds = DataLoader()

valid_x, valid_y = ds.train_gen().next()

In [7]:
from sklearn.metrics import confusion_matrix
import numpy as np

def kappa(y_true, y_pred):
    """
    Quadratic kappa score: http://www.kaggle.com/c/diabetic-retinopathy-detection/details/evaluation
    Implementaion mostly taken from: http://scikit-learn-laboratory.readthedocs.org/en/latest/_modules/skll/metrics.html

    :param y_true:
    :param y_pred:
    :return:
    """
    # Ensure that the lists are both the same length
    assert(len(y_true) == len(y_pred))

    # This rather crazy looking typecast is intended to work as follows:
    # If an input is an int, the operations will have no effect.
    # If it is a float, it will be rounded and then converted to an int
    # because the ml_metrics package requires ints.
    # If it is a str like "1", then it will be converted to a (rounded) int.
    # If it is a str that can't be typecast, then the user is
    # given a hopefully useful error message.
    # Note: numpy and python 3.3 use bankers' rounding.
    try:
        y_true = [int(np.round(float(y))) for y in y_true]
        y_pred = [int(np.round(float(y))) for y in y_pred]
    except ValueError as e:
        print("Kappa values must be integers or strings")
        raise e

    # Figure out normalized expected values
    min_rating = min(min(y_true), min(y_pred))
    max_rating = max(max(y_true), max(y_pred))

    # shift the values so that the lowest value is 0
    # (to support scales that include negative values)
    y_true = [y - min_rating for y in y_true]
    y_pred = [y - min_rating for y in y_pred]

    # Build the observed/confusion matrix
    num_ratings = max_rating - min_rating + 1
    observed = confusion_matrix(y_true, y_pred,
                                labels=list(range(num_ratings)))
    num_scored_items = float(len(y_true))

    # Build weight array if weren't passed one
    weights = np.empty((num_ratings, num_ratings))
    for i in range(num_ratings):
        for j in range(num_ratings):
            weights[i, j] = abs(i - j) ** 2

    hist_true = np.bincount(y_true, minlength=num_ratings)
    hist_true = hist_true[: num_ratings] / num_scored_items
    hist_pred = np.bincount(y_pred, minlength=num_ratings)
    hist_pred = hist_pred[: num_ratings] / num_scored_items
    expected = np.outer(hist_true, hist_pred)

    # Normalize observed array
    observed = observed / num_scored_items

    # If all weights are zero, that means no disagreements matter.
    k = 1.0
    if np.count_nonzero(weights):
        k -= (sum(sum(weights * observed)) / sum(sum(weights * expected)))

    return k

In [26]:
y_true = np.random.randint(0,4, 7000)
#y_hat = np.copy(y_true)
y_hat = np.random.randint(0,4, 7000)

In [27]:
kappa(y_true, y_hat)

-0.013730234170657774